In [2]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import nltk 
nltk.download('punkt')  # Download for sentence tokenization
nltk.download('stopwords')  # Download for stopword removal


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ecesoclab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ecesoclab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df = pd.read_csv('Reviews.csv')


In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import nltk
import string

# Initialize NLTK resources before your functions
nltk.download('punkt')
nltk.download('stopwords')

# Get the list of English stopwords
stop_words = set(nltk.corpus.stopwords.words('english'))
punctuation_table = str.maketrans('', '', string.punctuation)

def clean_html(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

def tokenize_and_remove_stopwords(text):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [w for w in tokens if w.lower() not in stop_words]
    return filtered_tokens

# Define the enhanced preprocessing function
def preprocess_text(text):
    text = text.lower()
    tokens = tokenize_and_remove_stopwords(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [word.translate(punctuation_table) for word in tokens]
    tokens = [word for word in tokens if word.strip()]
    return ' '.join(tokens)

# Apply preprocessing to your DataFrame
df['Processed_Summary'] = df['Summary'].apply(clean_html)
df['Processed_Summary'] = df['Processed_Summary'].apply(preprocess_text)  # Enhanced preprocessing

df['Processed_Text'] = df['Text'].apply(clean_html)
df['Processed_Text'] = df['Processed_Text'].apply(preprocess_text)  # Enhanced preprocessing


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ecesoclab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ecesoclab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df = df.reset_index(drop=True)


In [8]:
from sklearn.model_selection import train_test_split

# Split the data
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)


In [9]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [10]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Now you can use the tokenizer with padding
inputs = tokenizer("Example text", return_tensors="pt", padding=True)

In [11]:
from torch.utils.data import Dataset

class SummaryDataset(Dataset):
    def __init__(self, texts, summaries, tokenizer, max_length=512):
        self.texts = texts
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        summary = self.summaries[idx]
        encoding = tokenizer(
            text, summary,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        inputs = {
            'input_ids': encoding.input_ids.flatten(),
            'attention_mask': encoding.attention_mask.flatten(),
            'labels': encoding.input_ids.flatten()  # Using input_ids for labels for simplicity
        }
        return inputs

# Initialize datasets
train_dataset = SummaryDataset(train_df['Processed_Text'], train_df['Processed_Summary'], tokenizer)
test_dataset = SummaryDataset(test_df['Processed_Text'], test_df['Processed_Summary'], tokenizer)


In [ ]:
train_df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Processed_Summary,Processed_Text
82,83,B003ZFRKGO,A2VOZX7YBT0D6D,"Johnnycakes ""Johnnycakes""",15,15,5,1325635200,Forget Molecular Gastronomy - this stuff rocke...,I know the product title says Molecular Gastro...,forget molecular gastronomy stuff rockes coffe...,know product title says molecular gastronomy n...
254,255,B0048IC328,A1W867A8DSHFHC,no name,1,3,1,1339804800,Don't Waste Your Money,"I felt energized within five minutes, but it l...",nt waste money,felt energized within five minutes lasted 45 m...
922,923,B000ER6YO0,A3SFS8R1T6JJM9,PolyOWannaCracker,0,0,3,1318032000,Meh. My daughter eats most everything else......,I introduced my daughter to Earth's Best Organ...,meh daughter eats everything else nt thing,introduced daughter earth s best organic turke...
408,409,B001IZM8A6,AVI998S4IX2Y1,"Melissa N. Connor ""rare maven""",1,1,5,1257897600,A Fantastic & Healthy Product,A grande item ! Delicious fish with the ultim...,fantastic healthy product,grande item delicious fish ultimate flavor bes...
653,654,B002BCD2OG,A2ZE5ICI6LWAZ0,"Debra Schiff ""http://hereandthere123.blogspot...",27,27,5,1259193600,"Caramel flavor, excellent for baking and toppi...",If you want to find a good flavor substitute f...,caramel flavor excellent baking toppings tips ...,want find good flavor substitute maple syrup b...
...,...,...,...,...,...,...,...,...,...,...,...,...
106,107,B004K2IHUO,A1ZKFQLHFZAEH9,"S. J. Monson ""world citizen""",2,8,3,1236384000,disappointing,not what I was expecting in terms of the compa...,disappointing,expecting terms company s reputation excellent...
270,271,B000LKZK7C,A2VUB6YYTF234O,Anel Lopez,0,0,4,1302739200,Good product,"I only use raw sugar, it did seem a little sma...",good product,use raw sugar seem little smaller normal cryst...
860,861,B000VKYKTG,A1O9O18AE5HIOL,Sweet Tooth,2,2,3,1283472000,Chocolate was all melt,The chocolate on the stick was all melted and ...,chocolate melt,chocolate stick melted sticks stuck oneanother...
435,436,B000G6RYNE,A15USNEAJUXOSH,L. Schrank,0,0,5,1326067200,Delicious,"I love these chips, I buy the 24 pack once a m...",delicious,love chips buy 24 pack month bags right size s...


In [12]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Start training
trainer.train()


  2%|▏         | 10/567 [01:07<1:03:31,  6.84s/it]

{'loss': 7.4026, 'grad_norm': 216.87892150878906, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.05}


  4%|▎         | 20/567 [02:16<1:02:41,  6.88s/it]

{'loss': 6.5381, 'grad_norm': 208.92771911621094, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.11}


  5%|▌         | 30/567 [03:25<1:02:12,  6.95s/it]

...

 99%|█████████▉| 560/567 [1:09:06<00:52,  7.44s/it]

{'loss': 0.5064, 'grad_norm': 1.8076826333999634, 'learning_rate': 5.2238805970149255e-06, 'epoch': 2.96}


100%|██████████| 567/567 [1:09:53<00:00,  7.40s/it]

{'train_runtime': 4193.8343, 'train_samples_per_second': 0.539, 'train_steps_per_second': 0.135, 'train_loss': 0.9464302580192606, 'epoch': 3.0}


TrainOutput(global_step=567, training_loss=0.9464302580192606, metrics={'train_runtime': 4193.8343, 'train_samples_per_second': 0.539, 'train_steps_per_second': 0.135, 'train_loss': 0.9464302580192606, 'epoch': 3.0})

In [15]:
!pip install rouge-score



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
                                              0.0/133.7 kB ? eta -:--:--
     -------------------------------------- 133.7/133.7 kB 7.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24972 sha256=56b8578d9400395b7102fefe4a1b7541aebbacb0db77039c0398b244c4fbca36
  Stored in directory: c:\users\ecesoclab\appdata\local\pip\cache\wheels\5f\dd\89\461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\ecesoclab\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
from rouge_score import rouge_scorer

# Function to generate summaries
def generate_summary(model, tokenizer, text, max_length=100):
    try:
        input_ids = tokenizer.encode(text, return_tensors='pt')
        if input_ids.size(1) > max_length:
            print(f"Skipping generation for input of length {input_ids.size(1)} as it exceeds max_length of {max_length}.")
            return None

        summary_ids = model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True)
        return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None


# Function to compute ROUGE scores
def compute_rouge_scores(model, dataset, tokenizer):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    # Loop through the dataset
    for i in range(len(dataset)):
        text = dataset.texts[i]
        reference_summary = dataset.summaries[i]
        # Generate summary
        generated_summary = generate_summary(model, tokenizer, text)

        # Compute ROUGE scores
        score = scorer.score(reference_summary, generated_summary)
        scores['rouge1'].append(score['rouge1'].fmeasure)
        scores['rouge2'].append(score['rouge2'].fmeasure)
        scores['rougeL'].append(score['rougeL'].fmeasure)

    # Calculate average scores
    avg_scores = {
        'rouge1': sum(scores['rouge1']) / len(scores['rouge1']),
        'rouge2': sum(scores['rouge2']) / len(scores['rouge2']),
        'rougeL': sum(scores['rougeL']) / len(scores['rougeL'])
    }

    return avg_scores

# Example of how to call the compute_rouge_scores function
# This assumes you have a trained model and tokenizer ready, and a dataset prepared
avg_rouge_scores = compute_rouge_scores(model, test_dataset, tokenizer)
print("Average ROUGE Scores:", avg_rouge_scores)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Average ROUGE Scores: {'rouge1': 0.07907748833580741, 'rouge2': 0.017950841147095414, 'rougeL': 0.07279729568842672}


In [21]:
from rouge_score import rouge_scorer

def compute_rouge_scores(model, dataset, tokenizer, dataframe):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores_list = []

    # Loop through the dataset
    for i in range(len(dataset)):
        text = dataset.texts[i]
        reference_summary = dataset.summaries[i]

        # Generate summary
        generated_summary = generate_summary(model, tokenizer, text)

        # Compute ROUGE scores
        scores = scorer.score(reference_summary, generated_summary)
        
        # Create a detailed score dictionary
        detailed_scores = {
            'ROUGE-1 Precision': scores['rouge1'].precision,
            'ROUGE-1 Recall': scores['rouge1'].recall,
            'ROUGE-1 F1': scores['rouge1'].fmeasure,
            'ROUGE-2 Precision': scores['rouge2'].precision,
            'ROUGE-2 Recall': scores['rouge2'].recall,
            'ROUGE-2 F1': scores['rouge2'].fmeasure,
            'ROUGE-L Precision': scores['rougeL'].precision,
            'ROUGE-L Recall': scores['rougeL'].recall,
            'ROUGE-L F1': scores['rougeL'].fmeasure
        }

        # Append the score dictionary to the list
        scores_list.append(detailed_scores)

    # Append scores to the dataframe
    score_df = pd.DataFrame(scores_list)
    enhanced_df = pd.concat([dataframe.reset_index(drop=True), score_df], axis=1)
    return enhanced_df

# Call the function and pass the DataFrame
enhanced_df = compute_rouge_scores(model, test_dataset, tokenizer, test_df)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [24]:
# Save to CSV
enhanced_df.to_csv("detailed_rouge_scores_output.csv", index=False)

enhanced_df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,...,Processed_Text,ROUGE-1 Precision,ROUGE-1 Recall,ROUGE-1 F1,ROUGE-2 Precision,ROUGE-2 Recall,ROUGE-2 F1,ROUGE-L Precision,ROUGE-L Recall,ROUGE-L F1
0,927,B000ER6YO0,A2F0WNTW3QQZYS,Ruth,0,0,5,1298505600,One of our favorites,This is one of my son's favorite baby foods. ...,...,one son s favorite baby foods bit sweetness sw...,0.024390,1.000000,0.047619,0.000000,0.000000,0.000000,0.024390,1.000000,0.047619
1,631,B000G6RYNE,A1IVFBJA9KAI1M,Shane Martin,2,3,4,1191369600,Tasty!,The chips come in a large box with individuall...,...,chips come large box individually wrapped bags...,0.022727,1.000000,0.044444,0.000000,0.000000,0.000000,0.022727,1.000000,0.044444
2,683,B000G6MBX2,A1UDFKEKTO70NX,Tiffany,1,1,4,1245110400,tasty,These chips are perfect for snacking with or w...,...,chips perfect snacking without salsa texture b...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,515,B000G6RYNE,A16S0NQ3MWJXFL,ObsidianGT,0,1,5,1291248000,"Delicious, what else did you expect?","Seriously, now. Tasty, tasty, tasty. Fresh, to...",...,seriously tasty tasty tasty fresh problem pack...,0.015152,0.333333,0.028986,0.000000,0.000000,0.000000,0.015152,0.333333,0.028986
4,366,B00437JI8Q,A4TLHOUT1PHTL,Alison,0,0,5,1346198400,Can't find anywhere else!,My fiance loves these dark chocolate and nut g...,...,fiance loves dark chocolate nut granola bars g...,0.045455,0.200000,0.074074,0.000000,0.000000,0.000000,0.045455,0.200000,0.074074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,110,B001REEG6C,AY12DBB0U420B,Gary Peterson,0,0,5,1316390400,My Idea of a Good Diet Food.,I'm presently on a diet and I was at my Fresh ...,...,m presently diet fresh easy neighborhood groce...,0.050000,1.000000,0.095238,0.037975,1.000000,0.073171,0.050000,1.000000,0.095238
248,941,B000ER6YO0,A1ESDLEDR9Y0JX,A. Spencer,1,2,1,1310256000,the garbanzo beans in it give horrible gas,To be fair only one of my twins got gas from t...,...,fair one twins got gas horrible night screamin...,0.064516,0.800000,0.119403,0.016393,0.250000,0.030769,0.032258,0.400000,0.059701
249,78,B004X2KR36,A26M5O53PHZTKN,"Debs ""peanut""",0,0,5,1348185600,Taste great,These taste really good. I have been purchasin...,...,taste really good purchasing different brand s...,0.021739,0.500000,0.041667,0.000000,0.000000,0.000000,0.021739,0.500000,0.041667
250,85,B0019CW0HE,AK2CXHH9VRZ2A,I. GLENN,4,4,3,1313193600,INCREASED MY DOGS ITCHING,"Awesome dog food. However, when given to my ""B...",...,awesome dog food however given boston severe r...,0.088235,1.000000,0.162162,0.000000,0.000000,0.000000,0.058824,0.666667,0.108108
